In [5]:
import joblib
import pandas as pd

# 1. Carregar o modelo (Pipeline completo)
pipeline_carregado = joblib.load('./Model/XGB_Weight_19-20:04:30.pkl')

# 2. Carregar dados do Kaggle
df_kaggle = pd.read_csv('test.csv')
print(df_kaggle.columns.tolist())

# 3. Separar as colunas (Garantir que são as mesmas do treino)
# Se o seu treino não tinha a coluna 'ID', remova-a aqui
X_test_final = df_kaggle.drop(columns=["id"]) 

# 4. A MÁGICA DO PIPELINE:
# Ao chamar predict_proba, o Pipeline automaticamente:
# - Chama o preprocessor.transform() (Escala o Time e Amount)
# - Pula o sampler (SMOTE não é usado na predição, apenas no treino)
# - Chama o classifier.predict_proba()
y_probs = pipeline_carregado.predict_proba(X_test_final)[:, 1]

# 5. Gerar submissão
submission = pd.DataFrame({
    'ID': df_kaggle['ID'],
    'voted': y_probs
})
submission.to_csv('submissao_v1.csv', index=False)

['id', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']


ValueError: columns are missing: {'id'}

In [ ]:
best_model_name = "XGB_Weight"

y_pred = results[best_model_name].predict(X_test)
print(f"--- Relatório: {best_model_name} ---")
print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()